In [1]:
import pandas as pd
import os
import numpy as np
import requests
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime, timedelta
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data, process_weather_data
from get_electricity_prices import get_data
from get_weather_data import get_historical_weather, get_weather_forecast
from entsoe_data import fetch_historical_data, ensure_valid_series



In [2]:
load_dotenv()
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")
entose_api = os.getenv("ENTSOE_API")

os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
fs = project.get_feature_store() 
print(f"Connected to project: {project.name}")

2025-01-05 20:26:01,944 INFO: Initializing external client
2025-01-05 20:26:01,945 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 20:26:03,454 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164446
Connected to project: oskaralf


In [4]:
start_date = datetime.now().strftime('%Y-%m-%d')
end_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')

In [5]:
forecast = get_weather_forecast("Stockholm", "2022-11-01", "2025-01-03", 59.3294, 18.0687)

formatted_forecast_df = process_weather_data(forecast)
print(formatted_forecast_df.dtypes)

Coordinates 59.32889938354492°N 18.072357177734375°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
date                   datetime64[ns]
temperature_2m                float32
precipitation                 float32
snow_depth                    float32
pressure_msl                  float32
cloud_cover                   float32
wind_speed_10m                float32
wind_speed_100m               float32
wind_direction_10m            float32
wind_direction_100m           float32
city                           object
dtype: object


In [6]:
# price_fg = fs.get_feature_group(
#     name='electricity_price_data_3',
#     version=1,
# )
weather_fg = fs.get_feature_group(
    name='weather_data_3',
    version=1,
)
entsoe_fg = fs.get_feature_group(
    name='entsoe_data_3',
    version=1,
)

In [7]:

formatted_forecast_df['date'] = pd.to_datetime(formatted_forecast_df['date']).dt.tz_localize('UTC').dt.tz_convert(None)

In [8]:
print(formatted_forecast_df.isna().sum())

date                   0
temperature_2m         0
precipitation          0
snow_depth             0
pressure_msl           0
cloud_cover            0
wind_speed_10m         0
wind_speed_100m        0
wind_direction_10m     0
wind_direction_100m    0
city                   0
dtype: int64


In [9]:
formatted_forecast_df = formatted_forecast_df.dropna()
weather_fg.insert(formatted_forecast_df)

2025-01-05 20:27:11,278 INFO: 	5 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394619


Uploading Dataframe: 100.00% |██████████| Rows 168/168 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_3_1_offline_fg_materialization/executions


(Job('weather_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "temperature_2m",
           "min_value": -100.0,
           "max_value": 500.0
         },
         "meta": {
           "expectationId": 694496
         }
       },
       "result": {
         "observed_value": -9.399999618530273,
         "element_count": 168,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-05T07:27:11.000277Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be

In [8]:
# price = get_todays_data("SE3")
# price['time_start'] = price['time_start'].str.replace(r'\+\d{2}:\d{2}$', '', regex=True)
# print(price)
# print(price.dtypes)

In [10]:
entsoe_df = fetch_historical_data(entose_api, start_date, end_date)

[DEBUG] Successfully fetched load data for finland.
[DEBUG] Successfully fetched load data for norway.
[DEBUG] Successfully fetched load data for denmark.
[DEBUG] Successfully fetched cross-border flows SE3 to finland.
[DEBUG] Successfully fetched cross-border flows finland to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to norway.
[DEBUG] Successfully fetched cross-border flows norway to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to denmark.
[DEBUG] Successfully fetched cross-border flows denmark to SE3.


In [11]:
print(entsoe_df.isna().sum())

load_se3                                            0
load_finland                                        0
load_norway                                         0
load_denmark                                        0
prices                                              0
total_generation_Biomass                            0
total_generation_Fossil Gas                         0
total_generation_Fossil Hard coal                   0
total_generation_Fossil Oil                         0
total_generation_Hydro Run-of-river and poundage    0
total_generation_Other renewable                    0
total_generation_Solar                              0
total_generation_Waste                              0
total_generation_Wind Offshore                      0
total_generation_Wind Onshore                       0
flows_se3_to_finland                                0
flows_finland_to_se3                                0
flows_se3_to_norway                                 0
flows_norway_to_se3         

In [12]:

entsoe_df.columns = entsoe_df.columns.str.lower()
entsoe_df.columns = entsoe_df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
entsoe_df['date'] = pd.to_datetime(entsoe_df['date'])
entsoe_df['date'] = pd.to_datetime(entsoe_df['date']).dt.tz_localize('UTC').dt.tz_convert(None)
entsoe_df = entsoe_df.dropna()
print(entsoe_df.dtypes)
print(entsoe_df.tail(5))

load_se3                                                   float64
load_finland                                               float64
load_norway                                                float64
load_denmark                                               float64
prices                                                     float64
total_generation_biomass                                   float64
total_generation_fossil_gas                                float64
total_generation_fossil_hard_coal                          float64
total_generation_fossil_oil                                float64
total_generation_hydro_run_of_river_and_poundage           float64
total_generation_other_renewable                           float64
total_generation_solar                                     float64
total_generation_waste                                     float64
total_generation_wind_offshore                             float64
total_generation_wind_onshore                              flo

In [11]:
#print(entsoe_df['date'].tail(20))

In [16]:
# print(entose_api)
# entsoe_df = fetch_energy_data(entose_api, "2025-01-05", "2025-01-06")

# print(entsoe_df)

In [13]:
entsoe_fg.insert(entsoe_df)

2025-01-05 20:28:08,147 INFO: 	21 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394620


Uploading Dataframe: 100.00% |██████████| Rows 19/19 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: entsoe_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/entsoe_data_3_1_offline_fg_materialization/executions


(Job('entsoe_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "flows_norway_to_se3",
           "min_value": -0.1,
           "max_value": 10000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 694510
         }
       },
       "result": {
         "observed_value": 25.0,
         "element_count": 19,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-05T07:28:08.000146Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expec

In [11]:
# price_fg.insert(price)